## F1 state representation quality using multiple states  

In [1]:
from bbvi_infer_2l import * 
from f1 import *
%matplotlib inline
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import silhouette_score

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Read in data observations

In [2]:
year= 2022
gp = 'Japan'
event = 'Race'

session_event = ff1.get_session(year, gp, event)
session_event.load()
circuit_info = session_event.get_circuit_info()

req         WARNING 	DEFAULT CACHE ENABLED! (154.83 MB) /root/.cache/fastf1
core           INFO 	Loading data for Japanese Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '31', '44', '5', '14', '63', '6', '4', '3', '18', '22', '20', '77', '24', '47', '10'

In [3]:
# Train data
pos_winner= generate_full_laps(session_event.laps.pick_driver('VER'), 'VER')
## States 
states_train, zs_train = one_hot_states_multilap(pos_winner[['X','Y']].to_numpy(), circuit_info.corners[['X','Y']].to_numpy(), 
                                    pos_winner['LapNumber'].to_numpy())

/opt/conda/lib/python3.10/site-packages/fastf1/core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


In [4]:
## Standardize train data
import numpy.random as npr
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
track_norm = scaler.fit_transform(pos_winner[['X','Y']].to_numpy())
C = npr.randn(10, 2)  ## emission matrix
ys_train = (C @ track_norm.T).T + npr.randn(track_norm.shape[0], 10)*0.01

In [5]:
## Test data
pos_alt= generate_full_laps(session_event.laps.pick_driver('HAM'), 'HAM')
### States
states_test, zs_test = one_hot_states_multilap(pos_alt[['X','Y']].to_numpy(), circuit_info.corners[['X','Y']].to_numpy(), 
                                               pos_alt['LapNumber'].to_numpy())
### Observations
track_norm = scaler.transform(pos_alt[['X','Y']].to_numpy())
ys_test = (C @ track_norm.T).T + npr.randn(track_norm.shape[0], 10)*0.01

/opt/conda/lib/python3.10/site-packages/fastf1/core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


## GSD

In [6]:
gsd = []
for seed in range(3):  ## repeat tests
    torch.manual_seed(seed)  # Set PyTorch seed
    torch.cuda.manual_seed_all(seed)  # Set CUDA seed 
    accs = []
    for K in range(2, 19): ## twiddle states
        model = GenerativeSLDS(N=10, K=K, D=2, emission_model="gaussian")
        elbos, variational_z = fit_bbvi_schedule(model.to('cuda'), ys = torch.tensor(ys_train).to('cuda').float(), 
                                             num_iters=4000, learning=True, n_samples=10, base_lr=1e-2, warmup_iters=200, tau_max=0.99)
        zs = variational_z.sample_q_z(torch.tensor(ys_train).to('cuda').float().unsqueeze(0).expand(20, -1, -1), 0.99)
        zs2 = variational_z.sample_q_z(torch.tensor(ys_test).to('cuda').float().unsqueeze(0).expand(20, -1, -1), 0.99)
        pred_ys2=model.smooth(torch.tensor(ys_test).to('cuda').float(), zs2).mean(dim=0).detach().cpu().numpy() # [T, N]
        print(f"Test accuracy for {K} states:", train_metrics(ys_test, pred_ys2, None, k_max=0))
        Z_train = torch.mean(zs, dim=0).detach().cpu().numpy()
        Y_train = np.array(zs_train)
        Z_test = torch.mean(zs2, dim=0).detach().cpu().numpy()
        Y_test = np.array(zs_test)
        # Train KNN classifier
        knn = KNeighborsClassifier(n_neighbors=10, metric='euclidean')
        knn.fit(Z_train, Y_train)
        acc = knn.score(Z_test, Y_test)
        print(f"State quality for {K} states:", acc)
        accs.append(acc)  
    gsd.append(np.array(accs))
np.save('f1GSD.npy', np.array(gsd))

ELBO: -53369.6, LR: 0.01000, Tau: 0.990:   5%|▍         | 194/4000 [00:02<00:39, 96.23it/s] /opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
ELBO: 87695.8, LR: 0.00100, Tau: 0.990: 100%|██████████| 4000/4000 [00:41<00:00, 95.49it/s]


Test accuracy for 2 states: [0.974468207750306]
State quality for 2 states: 0.3969631236442516


ELBO: -3892.5, LR: 0.01000, Tau: 0.990:   5%|▍         | 190/4000 [00:02<00:40, 94.98it/s] /opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
ELBO: 250150.7, LR: 0.00100, Tau: 0.990: 100%|██████████| 4000/4000 [00:42<00:00, 94.97it/s]


Test accuracy for 3 states: [0.9973776521624671]
State quality for 3 states: 0.36969321351100093


ELBO: -9756.4, LR: 0.01000, Tau: 0.990:   5%|▍         | 199/4000 [00:02<00:41, 90.93it/s] /opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
ELBO: 272086.2, LR: 0.00100, Tau: 0.990: 100%|██████████| 4000/4000 [00:43<00:00, 91.23it/s]


...

ELBO: 30643.3, LR: 0.01000, Tau: 0.990:   5%|▍         | 194/4000 [00:03<01:16, 49.82it/s] /opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
ELBO: 272246.8, LR: 0.00100, Tau: 0.990:  89%|████████▉ | 3577/4000 [01:11<00:08, 50.05it/s]


Early stopping at iteration 3577.
Test accuracy for 15 states: [0.9977406713042987]
State quality for 15 states: 0.6645491168267741


ELBO: -37861.4, LR: 0.01000, Tau: 0.990:   5%|▍         | 195/4000 [00:04<01:24, 45.07it/s] /opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
ELBO: 220697.4, LR: 0.00100, Tau: 0.990: 100%|██████████| 4000/4000 [01:29<00:00, 44.91it/s]


Test accuracy for 16 states: [0.9959675368093786]
State quality for 16 states: 0.5037960954446855


ELBO: 21865.3, LR: 0.01000, Tau: 0.990:   5%|▍         | 195/4000 [00:04<01:30, 42.18it/s] /opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
ELBO: 275280.5, LR: 0.00100, Tau: 0.990:  78%|███████▊  | 3110/4000 [01:14<00:21, 41.70it/s]

## rSLDS

In [6]:
import ssm
from ssm.extensions.mp_srslds.transitions_ext import StickyRecurrentOnlyTransitions, StickyRecurrentTransitions

In [10]:
slds_acc = []
slds_sil = []
for seed in range(3):  ## repeat tests
    torch.manual_seed(seed)  # Set PyTorch seed
    torch.cuda.manual_seed_all(seed)  # Set CUDA seed 
    accs2 = []
    sils = []
    for K in range(2, 19):
        sro_trans=StickyRecurrentOnlyTransitions(K=K,D=2) 
        rslds = ssm.SLDS(N=10, K=K, D=2,
                 transitions=sro_trans,
                 dynamics="diagonal_gaussian",
                 emissions="gaussian",
                 single_subspace=True)
        rslds.initialize(ys_train)
        q_elbos_lem, q_lem = rslds.fit(ys_train, method="laplace_em", 
                                   variational_posterior="structured_meanfield",
                                   initialize=False, num_iters=100, alpha=0.0)
        q_lem_x = q_lem.mean_continuous_states[0]
        q_lem_z = rslds.most_likely_states(q_lem_x, ys_train)
        # plot_trajectory(q_lem_z, q_lem_x)
        elbos, q_lem2 = rslds.approximate_posterior(ys_test,
                                        method="laplace_em",
                                        variational_posterior="structured_meanfield",
                                        num_iters=20)
        q_lem_x2 = q_lem2.mean_continuous_states[0]
        q_lem_z2 = rslds.most_likely_states(q_lem_x2, ys_test)
        sil = silhouette_score(q_lem_x2, q_lem_z2)
        pred_ys2=rslds.smooth(q_lem_x2, ys_test)
        print(f"Test accuracy for {K} states:", train_metrics(ys_test, pred_ys2, None, k_max=0))
        print(f"Silhouette score at {K} states:", sil)
        Z_train = np.eye(K)[q_lem_z]
        Y_train = np.array(zs_train)
        Z_test = np.eye(K)[q_lem_z2]
        Y_test = np.array(zs_test)
        # Train KNN classifier
        knn = KNeighborsClassifier(n_neighbors=10, metric='euclidean')
        knn.fit(Z_train, Y_train)
        acc = knn.score(Z_test, Y_test)
        print(f"State quality for {K} states:", acc)
        accs2.append(acc)
        sils.append(sil)
    slds_acc.append(np.array(accs2))
    slds_sil.append(np.array(sils))
np.save('f1SLDS.npy', np.array(slds_acc))    
np.save('f1SLDS_sil.npy', np.array(slds_sil))   

ELBO: 353592.9: 100%|██████████| 20/20 [00:13<00:00,  1.48it/s]


Test accuracy for 2 states: [0.9993817874650348]
Silhouette score at 2 states: 0.04706478178738587
State quality for 2 states: 0.17260613572977998


ELBO: 358742.3: 100%|██████████| 20/20 [00:15<00:00,  1.32it/s]


Test accuracy for 3 states: [0.9993799412354623]
Silhouette score at 3 states: 0.024683241748336693
State quality for 3 states: 0.1789587852494577


ELBO: 362553.0: 100%|██████████| 20/20 [00:16<00:00,  1.18it/s]


...

ELBO: 366497.6: 100%|██████████| 20/20 [00:33<00:00,  1.65s/it]


Test accuracy for 9 states: [0.9993754959689438]
Silhouette score at 9 states: -0.09584790016588293
State quality for 9 states: 0.41439417415556246


ELBO: 366717.0: 100%|██████████| 20/20 [00:35<00:00,  1.80s/it]


Test accuracy for 10 states: [0.9993752361262912]
Silhouette score at 10 states: -0.11454486235643967
State quality for 10 states: 0.399907034397273


ELBO: 379816.8:  37%|███▋      | 37/100 [02:35<03:07,  2.98s/it]